# This generates a file containing all the binned MJ background histograms and plots the mjj histogram
## Input is
### Signal region path
### Multi jet control region path
### Sample used for Z-QCDjets

In [1]:
import ROOT as r
from ROOT import gStyle
import numpy as np
import ctypes
import os
from IPython.display import display 
import pandas as pd 

### Define the SR and MJ enhanced region ##########
signalRegionPath = "SR/"
multiJetRegionPath = "MJ-SR/"
##### SELECTING GENERATOR FOR SIGNAL SAMPLE
ztt_sample="Ztautau_SherpaRW"


# Erase previous MJ.root file
samples=[i for i in os.listdir(signalRegionPath) if '.root' in i]
print(samples)
if "MJ.root" in samples:
    os.system("rm "+signalRegionPath+"MJ.root")

%run "../../Scripts/Plotter.ipynb"

Welcome to JupyROOT 6.26/10
['Ztautau_SherpaRW.root', 'Wjets.root', 'Ztautau_MGRW.root', 'Zjets.root', 'Signal_Sherpa.root', 'Signal_PoPy.root', 'BG.root', 'Data.root', 'MC.root', 'VV.root', 'Higgs.root', 'Ztautau_Average.root', 'ttbar.root', 'singletop.root']


In [6]:
histos_no_rebin={
"n_bjets":['N b-jets'],
"lepiso":['Lepton Isolation'],
"n_jets_interval":['N jets gap'],
}

histos_rebin = {
"tau_pt":[[80.0,160.0],[20.0,40.0,170.0],20.0,'pT(#tau)'],
"lep_pt":[[80,160],[20,40,170],20,'pT(l)'],
"delta_phi":[[3.0],[0.3,0.2],0.3,'#Delta#phi(#tau,l)'],
"lep_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[[0.5],[0.5,0.5],0.5,'#eta(l)'],
"tau_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[[0.5],[0.5,0.5],0.5,'#eta(#tau)'],
"delta_R_taulep_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[[0.5],[0.5,0.5],0.5,'#DeltaR(#tau,l)'],
"delta_R_lepjet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[[0.5],[0.5,0.5],0.5,'#DeltaR(l,j)'],
"delta_R_taujet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[[0.5],[0.5,0.5],0.5,'#DeltaR(#tau,j)'],
"met_basic_dphi":[[100],[20,100],20,'MET'],
"delta_y":[[6.0],[1.0,4.0],1.0,'#Deltay_{jj}'],
"omega":[[-0.2,1.6],[2.8,0.3,1.4],0.3,'#Omega'],
"rnn_score_1p":[[0.25],[0.25,0.25],0.25,'jetRNN Score 1p'],
"rnn_score_3p":[[0.4],[0.2,0.1999],0.2,'jetRNN Score 3p'],
"ljet0_pt":[[400,600],[50,100,400],50,'pT(j_{1})'],
"ljet1_pt":[[150,350],[50,100,650],50,'pT(j_{2})'],
"pt_bal":[[0.15],[0.03,0.75],0.03,'pT balance'],
"Z_centrality":[[0.5],[0.1,0.5],0.1,'#xi(Z)'],
"mass_jj":[[1500,3000],[250,500,1000],250,'m_{jj}'],
"reco_mass_i":[[40,65,115,175],[40,25,10,15,65],10,'m_{#tau,l}(i)'],
"reco_mass_o":[[40,65,115,175],[40,25,10,15,65],10,'m_{#tau,l}(o)'],
"reco_mass_":[[40,65,115,175],[40,25,10,15,65],10,'m_{#tau,l}'],
"Z_pt_reco_i_basic_cuts_tpt":[[300],[50,100],50,'pT(Z)']
}

histos={}
histos.update(histos_no_rebin)
histos.update(histos_rebin)

def data_subtracted_plot(name,multiJetFile,path_to_samples=os.getcwd(),signal_name="Signal",data_name="Data",bg_names=[],histogram_dict=None):
    
    Data = ROOT.TFile.Open(path_to_samples+"/"+data_name+".root","READ")
    Signal = ROOT.TFile.Open(path_to_samples+"/"+signal_name+".root","READ")
    
    bg_files_list=[]
    for i in bg_names:
        bg_files_list.append(ROOT.TFile.Open(path_to_samples+"/"+i+".root","READ"))
    
    histogram_name = name
    print(histogram_name)
    data=Data.Get(histogram_name)
    signal=Signal.Get(histogram_name)
    bg_histograms_list = [i.Get(histogram_name) for i in bg_files_list]
    
    signal.SetDirectory(0)
    data.SetDirectory(0)
    for i in bg_histograms_list:
        i=i.SetDirectory(0)
    

    Signal.Close()
    Data.Close()
    [i.Close() for i in bg_files_list]
    
    s=data.GetXaxis().GetBinLowEdge(1)
    e=data.GetXaxis().GetBinUpEdge(data.GetNbinsX())
    
    try :
        rebining=biner(histogram_dict[histogram_name][0],histogram_dict[histogram_name][1],data)
        nb=len(rebining)-1

        signal=signal.Rebin(nb,"signal",rebining)
        data=data.Rebin(nb,"data",rebining)
        for i in range(0,len(bg_histograms_list)):
            bg_histograms_list[i]=bg_histograms_list[i].Rebin(nb,bg_names[i],rebining)

        if len(histogram_dict[histogram_name])>0:
            hist_list=[signal,data]+bg_histograms_list
            normalization(hist_list,histogram_dict[histogram_name][2])
            
    except IndexError: # No need to rebin
        pass
        
    data_subtracted=data.Clone()
    for i in bg_histograms_list:
        data_subtracted.Add(i,-1)
        
    mj = data_subtracted.Clone()    
    mj.Add(signal,-1)    
    
    if histogram_name=="mass_jj":
        
        canvas = ROOT.TCanvas("canvas")
        canvas.cd()
        
        data_subtracted.Draw()
        signal.SetLineColor(r.kRed)
        signal.Draw("same h")
        
        legend = ROOT . TLegend (0.45 ,0.80 ,0.85 ,0.95)
        legend . AddEntry ( signal ,"Z-QCDjj")
        legend . AddEntry ( data_subtracted ," Data Subracted ")
        legend.SetNColumns(3)
        gStyle.SetLegendBorderSize(0)
        legend . SetLineWidth (0)
        legend . Draw ("same")
        
        canvas.Print(histogram_name+".pdf")
        
    multiJetFile.WriteObject(mj,histogram_name)




## Create the MJ.root file and the mjj plot

In [7]:
multiJetFile =r.TFile.Open("MJ.root", "RECREATE")

for i in histos:
    print(i)
    data_subtracted_plot(i,multiJetFile,path_to_samples=multiJetRegionPath,signal_name=ztt_sample,data_name="Data",
                          bg_names=["Signal_Sherpa","VV","Wjets","ttbar","singletop","Higgs"],histogram_dict=histos)


multiJetFile.Close()


n_bjets
n_bjets
lepiso
lepiso
n_jets_interval
n_jets_interval
tau_pt
tau_pt
0.0
500.0
lep_pt
lep_pt
0.0
500.0
delta_phi
delta_phi
0.0
3.2
lep_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
lep_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
-2.5
2.5
tau_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
tau_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
-2.5
2.5
delta_R_taulep_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
delta_R_taulep_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
0.0
6.0
delta_R_lepjet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
delta_R_lepjet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
0.0
6.0
delta_R_taujet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
delta_R_taujet_basi

Info in <TCanvas::Print>: pdf file mass_jj.pdf has been created


In [4]:
print(histos)

{'n_bjets': ['N b-jets'], 'lepiso': ['Lepton Isolation'], 'n_jets_interval': ['N jets gap'], 'tau_pt': [[80.0, 160.0], [20.0, 40.0, 170.0], 20.0, 'pT(#tau)'], 'lep_pt': [[80, 160], [20, 40, 170], 20, 'pT(l)'], 'delta_phi': [[3.0], [0.3, 0.2], 0.3, '#Delta#phi(#tau,l)'], 'lep_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt': [[0.5], [0.5, 0.5], 0.5, '#eta(l)'], 'tau_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt': [[0.5], [0.5, 0.5], 0.5, '#eta(#tau)'], 'delta_R_taulep_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt': [[0.5], [0.5, 0.5], 0.5, '#DeltaR(#tau,l)'], 'delta_R_lepjet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt': [[0.5], [0.5, 0.5], 0.5, '#DeltaR(l,j)'], 'delta_R_taujet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt': [[0.5], [0.5, 0.5], 0.5, '#DeltaR(#tau,j)'], 'met_basic_dphi': [[100], [20, 100], 20, 'MET'], 'delt